In [1]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY')

In [2]:
pip install -q agno

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.0 MB/s eta 0:00:00


In [41]:
import pandas as pd

df = pd.read_csv('kpi.csv', parse_dates=['timestamp'])
df['hour'] = df['timestamp'].dt.hour

# Global storage for DataFrame
_dataframe_store = {'df': None}

In [42]:

from agno.tools import tool

@tool
def get_all_cells_prb_timeseries() -> dict:
    """
    Extracts Downlink PRB utilization values for ALL cells.

    Returns:
        Dictionary containing:
        - all_cells: list of all cell IDs
        - cells_data: dict with each cell's hourly PRB utilization
        - summary_stats: overall statistics
    """
    df = _dataframe_store['df']

    if df is None:
        return {"error": "No DataFrame loaded"}

    all_cells = df['cell_id'].unique().tolist()
    cells_data = {}

    for cell_id in all_cells:
        subset = df[df["cell_id"] == cell_id]

        if not subset.empty:
            prb_by_hour = subset.groupby("hour")["Downlink PRB Utilization"].mean().to_dict()

            cells_data[str(cell_id)] = {
                "prb_by_hour": {int(k): round(float(v), 2) for k, v in prb_by_hour.items()},
                "avg_prb": round(float(subset["Downlink PRB Utilization"].mean()), 2),
                "max_prb": round(float(subset["Downlink PRB Utilization"].max()), 2),
                "min_prb": round(float(subset["Downlink PRB Utilization"].min()), 2)
            }

    # Calculate summary statistics
    summary_stats = {
        "total_cells": len(all_cells),
        "overall_avg_prb": round(float(df["Downlink PRB Utilization"].mean()), 2),
        "overall_max_prb": round(float(df["Downlink PRB Utilization"].max()), 2),
        "overall_min_prb": round(float(df["Downlink PRB Utilization"].min()), 2)
    }

    return {
        "all_cells": all_cells,
        "cells_data": cells_data,
        "summary_stats": summary_stats
    }

# Store the DataFrame
_dataframe_store['df'] = df



In [43]:
# Create agent with enhanced instructions
from agno.agent import Agent
from agno.models.google import Gemini

prb_agent = Agent(
    name="PRB Network Optimization Agent",
    role="Telecom Network PRB utilization expert and optimization specialist",
    model=Gemini(
        id="gemini-2.0-flash",
        api_key=userdata.get('GOOGLE_API_KEY')
    ),
    tools=[get_all_cells_prb_timeseries],
    instructions="""
    You are an expert telecom network optimizer analyzing PRB utilization across all cells.

    Your task:
    1) Call get_all_cells_prb_timeseries() to retrieve data for ALL cells

    2) For EACH cell, analyze:
       - Peak PRB hours (highest utilization periods)
       - Low traffic hours (lowest utilization periods)
       - Average daily utilization pattern
       - Congestion indicators (PRB > 70% indicates congestion)
       - Unusual spikes or anomalies

    3) Determine optimal THRESHOLD values:
       - Calculate a PRB threshold for each cell below which it can be switched off
       - Consider: minimum PRB values, traffic patterns, redundancy requirements
       - Thresholds should typically be between 10-30% PRB utilization
       - Account for time-of-day patterns (e.g., nighttime vs daytime)

    4) Provide a comprehensive report with:
       - Summary of all cells analyzed
       - Individual cell analysis with peak/low hours
       - **CRITICAL**: A table/list showing each cell with its recommended switch-off threshold
       - Optimization recommendations (load balancing, cell switching strategies)
       - Potential energy savings estimates

    5) Format the threshold recommendations clearly as:
       Cell ID | Recommended Threshold (%) | Rationale

    Be specific with numerical thresholds for each cell based on the actual data patterns.
    """,
    markdown=True,
)

# Run the agent


In [44]:
result = prb_agent.run(
    """Analyze PRB utilization for all cells in the network.

    For each cell, determine:
    1. Traffic patterns and peak/low hours
    2. Optimal PRB threshold value for switching off the cell during low utilization
    3. Provide a clear table with cell IDs and their recommended thresholds

    Be specific with the threshold values based on the actual data."""
)

print(result.content)

Okay, I have analyzed the PRB utilization data for all 10 cells. Here's a summary of my findings and recommendations:

**Summary of Analysis**

The data reveals varying PRB utilization patterns across the cells. Some cells exhibit consistently high utilization (e.g., Cell_01, Cell_05), indicating potential congestion. Others have lower average utilization with distinct peak and low traffic hours (e.g., Cell_02, Cell_03, Cell_04, Cell_08). Most cells show a general trend of higher utilization during daytime/evening hours and lower utilization during late-night/early-morning hours.

**Individual Cell Analysis**

Here's a breakdown of each cell's traffic patterns:

*   **Cell\_01:**
    *   Peak Hours: 17:00 - 21:00
    *   Low Traffic Hours: 0:00 - 5:00
    *   Average PRB: 52.15%
    *   Congestion: High Utilization.  Consider load balancing.

*   **Cell\_02:**
    *   Peak Hours: 17:00 - 21:00
    *   Low Traffic Hours: 0:00 - 5:00
    *   Average PRB: 22.43%

*   **Cell\_03:**
    *  

In [38]:

from agno.tools import tool

@tool
def get_cell_prb_timeseries(cell_id: str) -> dict:
    """
    Extracts Downlink PRB utilization values for a single cell.

    Args:
        cell_id: The cell identifier to analyze (e.g., 'Cell_07')

    Returns:
        Dictionary containing cell_id and prb_by_hour mapping
    """
    df = _dataframe_store['df']

    if df is None:
        return {"error": "No DataFrame loaded"}

    subset = df[df["cell_id"] == cell_id]

    if subset.empty:
        return {"error": f"No data found for cell {cell_id}"}

    prb_by_hour = subset.groupby("hour")["Downlink PRB Utilization"].mean().to_dict()

    return {
        "cell_id": str(cell_id),
        "prb_by_hour": {int(k): float(v) for k, v in prb_by_hour.items()}
    }

# Store the DataFrame BEFORE creating the agent
_dataframe_store['df'] = df

In [39]:
from agno.agent import Agent
from agno.models.google import Gemini

prb_agent = Agent(
    name="PRB Analyzer Agent",
    role="Telecom PRB utilization expert",
    model=Gemini(
        id="gemini-2.0-flash",
        api_key=userdata.get('GOOGLE_API_KEY')
    ),
    tools=[get_cell_prb_timeseries],
    instructions="""
    You analyze PRB (Physical Resource Block) utilization trends for telecom cells.

    When asked to analyze a cell:
    1) Call get_cell_prb_timeseries with ONLY the cell_id parameter (e.g., 'Cell_07')
    2) Analyze the returned hourly PRB data to identify:
       - Peak PRB hours (highest utilization)
       - Low traffic hours (lowest utilization)
       - Overall traffic trend throughout the day
       - Any congestion indicators (PRB > 70%)
       - Unusual spikes or anomalies
    3) Provide actionable optimization recommendations

    Example: get_cell_prb_timeseries(cell_id='Cell_07')
    """,
    markdown=True,
)


In [40]:
result = prb_agent.run(
    "Analyze PRB utilization for Cell_07 using the get_cell_prb_timeseries function."
)
print(result.content)


Okay, I have the PRB utilization data for Cell_07. Here's my analysis:

**Overall Traffic Trend:**

The PRB utilization appears to be relatively consistent throughout the day, with a slight increase during the late afternoon and early evening hours.

**Peak PRB Hours:**

*   Hours with the highest utilization: 18:00 (6 PM) with 43.43% and 19:00 (7 PM) with 43.09%.

**Low Traffic Hours:**

*   Hours with the lowest utilization: 0:00 (12 AM) with 39.67%

**Congestion Indicators:**

*   The PRB utilization remains below 70% throughout the day, so there are no immediate congestion concerns.

**Unusual Spikes or Anomalies:**

*   No significant spikes or anomalies are apparent in the data. The utilization fluctuates within a relatively narrow range.

**Optimization Recommendations:**

*   **Capacity Planning:** While there is no immediate congestion, the cell consistently operates around 40% utilization. Monitor the growth of traffic to proactively plan for capacity upgrades if the utilizat